# LeNet 5

En este Notebook se va a implementar la red convolucional **LeNet 5** en el dataset de nubes creado anteriormente.

In [48]:
from sklearn.externals import joblib

import numpy as np
import pandas as pd
import sklearn.utils
import matplotlib.pyplot as plt

%matplotlib inline

Primero cargamos los datos desde el disco

In [49]:
cloud_pd = joblib.load('data/clouds.pkl')
cloud_pd = sklearn.utils.shuffle(cloud_pd)

Dividimos el dataset en sets de train y test.

In [50]:
batchsize = 32
nData = cloud_pd.shape[0]
nData_training = int(nData * 2/3)

# Batches number for training
batches = int(nData_training / batchsize)
nData_testing = nData - nData_training

print('- Number Data training: {}'.format(nData_training))
print('- Number batches: {}'.format(batches))
print('- Number Data testing: {}'.format(nData_testing))

- Number Data training: 13312
- Number batches: 416
- Number Data testing: 6656


Crear una lookup_table con las etiquetas de las nubes y generar sus indices para el dataset.

In [51]:
cloud_lookup_table, cloud_pd.cloud_type = np.unique(cloud_pd.cloud_type.values,
                                                    return_inverse=True)

print('- Clouds Lookup-table: ', cloud_lookup_table)

- Clouds Lookup-table:  ['sky']


In [52]:
cloud_x = cloud_pd.iloc[:nData_training, 2:]
cloud_y = cloud_pd.iloc[:nData_training, 0]

cloud_test_x = cloud_pd.iloc[nData_training:, 2:]
cloud_test_y = cloud_pd.iloc[nData_training:, 0]

Importar paquetes necesarios de **pyTorch** y convertir los datos a tensores.

In [53]:
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.optim

In [54]:
# Training
cloud_x = cloud_x.values.reshape((batches, batchsize, 
                                  32, 32, 3)).astype('uint8')
cloud_x = np.transpose(cloud_x, axes=(0, 1, 4, 2, 3))

cloud_y = cloud_y.values.reshape((batches, batchsize))

# Testing
cloud_test_x = cloud_test_x.values.reshape((cloud_test_x.shape[0],
                                            32, 32, 3)).astype('uint8')
cloud_test_x = np.transpose(cloud_test_x, axes=(0, 3, 1, 2))

cloud_test_y = cloud_test_y.values

# Convert to tensors
cloud_x = Variable(torch.Tensor(cloud_x).contiguous().cuda())
cloud_y = Variable(torch.LongTensor(cloud_y).contiguous().cuda())
cloud_test_x = Variable(torch.Tensor(cloud_test_x).contiguous().cuda())

print('- Cloud_x {}'.format(cloud_x.shape))
print('- Cloud_y {}'.format(cloud_y.shape))
print('- Cloud_test_x {}'.format(cloud_test_x.shape))
print('- Cloud_test_y {}'.format(cloud_test_y.shape))

- Cloud_x torch.Size([416, 32, 3, 32, 32])
- Cloud_y torch.Size([416, 32])
- Cloud_test_x torch.Size([6656, 3, 32, 32])
- Cloud_test_y (6656,)
